In [1]:
import pandas as pd
import prepare
import model
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_json('data1.json')
df = prepare.prep_repos(df)

In [3]:
target = 'language_reduced'

**Split the Data** into three separate samples to avoid data leakage while fitting and evaluating machine learning models.

In [4]:
# split the data using sklearn's train_test_split inside the split_data function
train, validate, test = prepare.split_data(df, target)

train	 n = 60
validate n = 27
test	 n = 22


# Modeling

The following code initializes the infrastructure for storing information about our models' performance

In [5]:
# initialize the model_number variable at 0
model_number = 0 
# initialize an empty dataframe
model_results = pd.DataFrame()

### Baseline

Here, we establish baseline predictions based on the most frequently ocurring target class and evaluate the baseline accuracy.

In [6]:
# create baseline predictions using the mode and get accuracy score with sklearn.metrics inside run_baseline()
model_number, model_results = model.run_baseline(train[target], model_number, model_results)

### Machine Learning Classsifiers

Here, we create, fit and evaluate the following types of machine learning classification models, using our train and validate samples. 

- Decision Tree
- Random Forest
- Multinomial Naive Bayes
- Complement Naive Bayes

Each model is run using two types of feature preprocessing:

- Term Frequency - Inverse Document Frequency (TF/IDF) Vectorization
- Count Vectorization (CV) AKA "Bag of Words"

For each type of model and type of preprocessing, the other associated hyperparameters were also varied, creating a total of 68 unique models with varying performance.

In [7]:
# user defined functions which create sklearn models with varying features and hyperparameters and store
# information about the models and their performance
model_number, model_results = model.run_decision_tree(train, validate, target, model_number, model_results)
model_number, model_results = model.run_random_forest(train, validate, target, model_number, model_results)
model_number, model_results = model.run_naive_bayes(train, validate, target, model_number, model_results)

### Results

Below, you can see the resulting accuracy scores on both the train and validate set for each model created.

In [8]:
model.display(model_results)

model_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,baseline
sample_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train,0.833333,0.783333,0.883333,0.800000,0.883333,0.883333,0.933333,0.916667,0.95000,0.950000,0.966667,0.966667,0.983333,0.983333,0.983333,0.983333,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.933333,0.966667,0.983333,0.966667,0.666667,0.950000,0.983333,0.966667,0.666667,0.933333,0.933333,0.966667,0.666667,0.883333,0.900000,0.966667,0.666667,0.850000,0.866667,0.933333,0.666667
validate,0.666667,0.629630,0.629630,0.481481,0.629630,0.518519,0.629630,0.555556,0.62963,0.592593,0.666667,0.592593,0.629630,0.592593,0.666667,0.592593,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.703704,0.703704,0.666667,0.703704,0.703704,0.740741,0.666667,0.703704,0.703704,0.740741,0.666667,0.666667,0.703704,0.740741,0.666667,0.666667,0.703704,0.740741,NaN


### Best Model

Here, we choose the model which performed with the highest level of **accuracy** on the validate sample. We choose accuracy as our evaluation metric because in this multi-class classifier, we have no reason to favor any one of the target classes over any other.

In [9]:
# obtain the model numbers with the highest accuracy on validate
best = model.get_best(model_results)
# display info about those models
model_results[model_results.model_number.isin(best)]

,model_number,model_type,sample_type,accuracy,feature_type,max_depth,min_samples_leaf,alpha
111,56,ComplementNB,train,0.966667,CV/BOW,NaN,NaN,0.5
112,56,ComplementNB,validate,0.740741,CV/BOW,NaN,NaN,0.5
119,60,ComplementNB,train,0.966667,CV/BOW,NaN,NaN,1.0
120,60,ComplementNB,validate,0.740741,CV/BOW,NaN,NaN,1.0
127,64,ComplementNB,train,0.966667,CV/BOW,NaN,NaN,1.5
128,64,ComplementNB,validate,0.740741,CV/BOW,NaN,NaN,1.5
135,68,ComplementNB,train,0.933333,CV/BOW,NaN,NaN,2.0
136,68,ComplementNB,validate,0.740741,CV/BOW,NaN,NaN,2.0


We find that we have 4 models with identical accuracy scores on the validate sample, so we will choose Model #68 as our best model, since it has the smallest dropoff in performance between train and validate.

### Final Test

Here, we recreate model #6 and evaluate it on our test sample, which approximates how it might perform when used on additional READMEs which were not included in our samples.

In [10]:
model.test_model_68(train, test, target)

0.5909090909090909

### Modeling - Takeaways

Our best model achieves only 59% accuracy on the test sample, which is actually 7 points lower than our baseline performance of 66%, and 15 points lower than the 74% accuracy on validate. 

Such a large variation in scores may be due simply to the small sample size of READMEs included in this analysis. A larger dataset would likely prove useful in creating a better model.

In [11]:
df.shape

(109, 7)